## Classes definition

In [ ]:
class Supplier:
    def __init__(self):
        self.µ_price_high_complexity = µ_price_high_complexity
        self.σ_price_high_complexity = σ_price_high_complexity
        self.µ_price_medium_complexity = µ_price_medium_complexity
        self.σ_price_medium_complexity = σ_price_medium_complexity
        self.µ_price_low_complexity = µ_price_low_complexity
        self.σ_price_low_complexity = σ_price_low_complexity


class Part_Number:
    def __init__(self):
        pass

class RFQ:
    def __init__(self):
        pass